## 2+.2 深度学习计算
> 此小节重点来讲解`Pytorch`中如何使用自定义的层与块。以及训练过程当中可做的优化。

In [44]:
%matplotlib inline
import torch
from d2l import torch as d2l
from torch import nn
import torch.nn.functional as F

### 层和快

首先我们先来回顾一下多层感知机(MLP)

In [45]:
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0941,  0.1131, -0.0657, -0.0093, -0.1215, -0.0907,  0.0767,  0.1591,
          0.2196, -0.0111],
        [ 0.0147,  0.1565, -0.0790,  0.0503,  0.0081, -0.1217,  0.1359,  0.1884,
          0.2071, -0.1020]], grad_fn=<AddmmBackward>)

下面我们来自定义块:
* 重新定义上面的`MLP

In [46]:
class MLP(nn.Module):
    """自定义块"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.1548,  0.0942, -0.1172,  0.0299,  0.1492, -0.0315,  0.1105,  0.2187,
         -0.0263, -0.0024],
        [ 0.1590,  0.2487, -0.1761, -0.0142,  0.0508, -0.1185,  0.2809,  0.2769,
         -0.0126,  0.0175]], grad_fn=<AddmmBackward>)

* 自定义

In [47]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super(MySequential, self).__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 20))

net(X)

tensor([[ 0.1006,  0.3091,  0.0457, -0.3274, -0.1046, -0.0743,  0.0062,  0.1815,
          0.2542,  0.1076,  0.0902, -0.0046,  0.0542,  0.0254, -0.1291,  0.0353,
         -0.0924, -0.0355, -0.0715,  0.0083],
        [ 0.1102,  0.2208,  0.0295, -0.1659, -0.0512, -0.1295,  0.0791,  0.1761,
          0.0168,  0.0048,  0.1624, -0.0684,  0.0283, -0.0011, -0.0404,  0.0676,
         -0.2070,  0.0503,  0.0864, -0.1108]], grad_fn=<AddmmBackward>)

* 自定义`forward`函数

In [48]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    # 做自己想做的事情
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.3020, grad_fn=<SumBackward0>)

* 混合其他组合块来使用

In [49]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

net = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
net(X)

tensor(-0.2653, grad_fn=<SumBackward0>)

自定义层:
* 无参数

In [50]:
# 无参数自定义层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X -X.sum()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-14., -13., -12., -11., -10.])

合并到更加复杂网络当中：

In [51]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-13.3828, grad_fn=<MeanBackward0>)

* 带参数

In [52]:
class MyLinear(nn.Module):
    def __init__(self, in_units, out_units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, out_units))
        self.bias = nn.Parameter(torch.randn(out_units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.5690,  0.5214,  0.4789],
        [-2.0068,  0.6800,  1.4595],
        [ 0.0995, -1.0156,  0.7387],
        [ 1.6798,  0.0154, -0.4543],
        [-1.0243,  0.4657,  0.7400]], requires_grad=True)

In [53]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

### 参数管理

In [54]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size=(2,4))
net(X)

tensor([[-0.3830],
        [-0.3076]], grad_fn=<AddmmBackward>)

* 参数访问（拿出权重）

In [55]:
print(net[2].state_dict())  # 最后输出层

OrderedDict([('weight', tensor([[ 0.1063, -0.2481, -0.2209, -0.1651, -0.2057, -0.1347,  0.2593,  0.1635]])), ('bias', tensor([-0.3103]))])


* 目标参数

In [57]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

net[2].weight.grad == None

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3103], requires_grad=True)
tensor([-0.3103])


True

* 一次性访问所有参数

In [58]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


* 通过名字获取参数

In [59]:
net.state_dict()['2.bias'].data

tensor([-0.3103])

* 嵌套块

In [61]:
# 嵌套块
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0818],
        [-0.0819]], grad_fn=<AddmmBackward>)

In [62]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


* 内置初始化
> 实际操作上不可以weight全部初始化为同一个数的

In [63]:
# 内置初始化
def init_weight(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_weight)
net[0].weight.data[0], net[0].bias.data

(tensor([-0.0076,  0.0047,  0.0052,  0.0031]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [64]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]  # 实际操作上不可以weight全部初始化为同一个数的

(tensor([1., 1., 1., 1.]), tensor(0.))

* 对某些块应用不同初始化方法, 使用$Xavier$初始化：

In [65]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.4579,  0.2206,  0.2903, -0.0741],
        [-0.0535,  0.1398, -0.4323, -0.1593],
        [ 0.4775,  0.0352, -0.1122, -0.0338],
        [-0.0536, -0.5981,  0.2187, -0.6939],
        [-0.0698,  0.4790, -0.5140,  0.6544],
        [-0.4063,  0.1822, -0.4695, -0.4123],
        [ 0.6459, -0.3181,  0.4643, -0.6863],
        [ 0.4404, -0.4913, -0.6809, -0.3302]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


* 共享权重

In [66]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


### 读写文件
> 本部分没有什么过度的技巧，具体通过理解代码来掌握`Pytorch`如何读写文件。

In [67]:
x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

* 存放其他数据

In [68]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

* 字典映射张量

In [69]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

* 加载和保存模型参数

In [71]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
x = torch.randn((2, 20))
Y = net(x)
# 保存
torch.save(net.state_dict(), 'mlp.params')

# 取参数（字典到tensor的映射）
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

# 验证
Y_clone = clone(x)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 使用GPU
默认情况下都是在CPU上进行运算的。

我们先看看如何使用单个NVIDIA GPU进行计算。 首先，确保你至少安装了一个NVIDIA GPU。 然后，下载[NVIDIA驱动和CUDA](https://developer.nvidia.com/cuda-downloads) 并按照提示设置适当的路径。 当这些准备工作完成，就可以使用`nvidia-smi`命令来查看显卡信息。

**注意**:由于我的机子上是没有NVIDIA的GPU的，因此，运行下面的代码会报错。

In [72]:
!nvidia-smi

Sat Mar 26 22:29:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.65       Driver Version: 511.65       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P0    N/A /  N/A |      0MiB /  2048MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [80]:
torch.device('cpu')
torch.cuda.device('cuda')  # 第0个GPU
torch.cuda.device('cuda:1')  # 第1个GPU

# 查询可用gpu的数量
torch.cuda.device_count()

# 根据索引号查看GPU名字:
torch.cuda.get_device_name(0)


# 允许我们在请求GPU不存在的时候用CPU来跑代码
def try_gpu(i=0):
    """如果存在，则返回gpu(i), 否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


def try_all_gpu():
    """ 返回所有可用GPU，没有可用就返回CPU"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


try_gpu(), try_gpu(10), try_all_gpu()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

* 在GPU上闯将Tensor

In [81]:
x = torch.tensor([1, 2, 3])
x.device  # CPU上

# 存储在GPU上
X = torch.ones(2, 3, device=try_gpu())
X

# 第二个GPU上创建一个随机张量
Y = torch.rand(2, 3, device=try_gpu(1))
Y

# 运算, 要在同一个GPU上, GPU向CPU传数据非常慢的
Z = X.cuda(1)
print(X)
print(Z)
Y + Z
Z.cuda(1) is Z  # True

AssertionError: Torch not compiled with CUDA enabled

* 将神经网络模型放入GPU

In [82]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cpu')